In [1]:
import tensorflow as tf
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import pathlib
from PIL import Image
import numpy
import os
import sys
from keras import layers



In [6]:
#set working directory
os.chdir(r'C:\Users\ethan\Desktop\mandarin_recognition_code2')

In [ ]:
train_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Train\Train')
test_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Test\Test')

'\ntrain_chinese = os.listdir(train_dir)\ntest_chinese = os.listdir(test_dir)\nlen(train_chinese)\nlen(test_chinese)\n'

In [ ]:
#make tensorflow dataset from image directory (if not already saved on local disk)


train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    seed=42,
    validation_split = 0.2,
    subset = 'both',
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)



Found 3223042 files belonging to 7330 classes.
Using 2578434 files for training.
Using 644608 files for validation.
Found 776523 files belonging to 7330 classes.


In [ ]:
#if it is first time generating dataset, save it to local disk
tf.data.Dataset.save(train_dataset, 'train_dataset')
tf.data.Dataset.save(val_dataset, 'val_dataset')
tf.data.Dataset.save(test_dataset, 'test_dataset')

In [8]:
#load dataset from local disk
train_dataset = tf.data.Dataset.load('train_dataset')
test_dataset = tf.data.Dataset.load('test_dataset')
val_dataset = tf.data.Dataset.load('val_dataset')

In [ ]:
#Check shape of dataset
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (32, 56, 56, 1)
Labels shape: (32, 7330)


In [ ]:
print("starting rescaling")

rescaling_layer = layers.Rescaling(scale=1./255)
train_dataset = train_dataset.map(lambda x, y: (rescaling_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (rescaling_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (rescaling_layer(x), y))


starting rescaling


In [ ]:

print("starting normalization")
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

'''
train_dataset_for_adapt = train_dataset.map(lambda x, y: x)
test_dataset_for_adapt = test_dataset.map(lambda x, y: x)
val_dataset_for_adapt = val_dataset.map(lambda x, y: x)

normalization_layer_train = layers.Normalization(axis=-1)
normalization_layer_test = layers.Normalization(axis=-1)
normalization_layer_val = layers.Normalization(axis=-1)
'''

def calculate_mean_variance_incrementally(dataset):
    # Initialize variables for mean, variance, and sample count
    mean_accumulator = 0.0
    variance_accumulator = 0.0
    sample_count = 0
    
    for images, _ in dataset:
        # Flatten images to process all pixels
        flattened_images = tf.reshape(images, (images.shape[0], -1))
        batch_size, num_pixels = flattened_images.shape
        
        # Update total sample count
        sample_count += batch_size * num_pixels
        
        # Calculate batch mean and variance
        batch_mean = tf.reduce_mean(flattened_images)
        batch_variance = tf.reduce_mean(tf.square(flattened_images - batch_mean))
        
        # Incrementally update the accumulated mean and variance
        delta = batch_mean - mean_accumulator
        mean_accumulator += delta / sample_count * (batch_size * num_pixels)
        variance_accumulator += batch_variance * (batch_size * num_pixels)
    
    # Finalize mean and variance calculation
    overall_mean = mean_accumulator
    overall_variance = variance_accumulator / sample_count
    
    return overall_mean.numpy(), overall_variance.numpy()


train_mean, train_variance = calculate_mean_variance_incrementally(train_dataset)
test_mean, test_variance = calculate_mean_variance_incrementally(test_dataset)
val_mean, val_variance = calculate_mean_variance_incrementally(val_dataset)

normalization_layer_train = layers.Normalization(axis=-1, mean=train_mean, variance=train_variance)
normalization_layer_test = layers.Normalization(axis=-1, mean=test_mean, variance=test_variance)
normalization_layer_val = layers.Normalization(axis=-1, mean=val_mean, variance=val_variance)

'''
print("starting adapt")
normalization_layer_train.adapt(train_dataset_for_adapt)
normalization_layer_test.adapt(test_dataset_for_adapt)
normalization_layer_val.adapt(val_dataset_for_adapt)

print("finished adapt")

'''
'''
train_dataset = train_dataset.map(lambda x, y: (normalization_layer_train(x), y), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer_test(x), y), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: (normalization_layer_val(x), y), num_parallel_calls=tf.data.AUTOTUNE)

'''

starting normalization


KeyboardInterrupt: 

In [ ]:
print("starting augmentation")

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))
test_dataset = test_dataset.map(lambda x, y: (data_augmentation(x), y))
val_dataset = val_dataset.map(lambda x, y: (data_augmentation(x), y))

starting augmentation


ValueError: in user code:

    File "C:\Users\ethan\AppData\Local\Temp\ipykernel_39772\3991776876.py", line 8, in None  *
        lambda x, y: (data_augmentation(x), y)
    File "c:\Users\ethan\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\ethan\anaconda3\envs\tf\lib\site-packages\keras\layers\preprocessing\image_preprocessing.py", line 444, in call
        raise ValueError(

    ValueError: Exception encountered when calling layer "random_flip_2" "                 f"(type RandomFlip).
    
    Image augmentation layers are expecting inputs to be rank 3 (HWC) or 4D (NHWC) tensors. Got shape: (None, None, 56, 56, 1)
    
    Call arguments received by layer "random_flip_2" "                 f"(type RandomFlip):
      • inputs=tf.Tensor(shape=(None, None, 56, 56, 1), dtype=float32)
      • training=True


In [ ]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


model = tf.keras.Sequential()
#model.add(normalization_layer_train)
model.add(layers.Conv2D(filters=80, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(56,56,1)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=160, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=320, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters= 640, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dense(units=200, activation='relu'))
model.add(layers.Dense(units=7330, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 80)        800       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 80)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 160)       115360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 160)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 320)       461120    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 320)        0

In [ ]:
predict_img = tf.keras.utils.load_img(r"C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Test\Test\中\1.png", target_size=(56,56))
img_array = tf.keras.utils.img_to_array(predict_img)
img_array = tf.expand_dims(img_array, 0)
img_array = img_array/255

NameError: name 'tf' is not defined

In [ ]:
# Convert the image to grayscale
img_array_gray = tf.image.rgb_to_grayscale(img_array)

# Normalize the grayscale image
img_array_gray = img_array_gray / 255.0

# Predict the class
prediction = model.predict(img_array_gray)
predicted_class = numpy.argmax(prediction, axis=-1)

print(predicted_class)

1/1 [==============================] - 0s 442ms/step
[58]


In [ ]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in train_dataset:   # use dataset.unbatch() with repeat
    # append true labels
    y_true.append(label_batch)
    # compute predictions
    label_batch = tf.image.rgb_to_grayscale(image_batch)
    label_batch = tf.reshape(label_batch, (label_batch.shape[0], -1))
    label_batch = map(label_batch, layers.Rescaling(.1/255))
    preds = model.predict(image_batch)
    # append predicted labels
    y_pred.append(numpy.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:GPU:0}} Matrix size-incompatible: In[0]: [100352,1], In[1]: [3,1] [Op:MatMul]

In [ ]:
class_names = train_dataset.element_spec
print(class_names)

(TensorSpec(shape=(None, 56, 56, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7330), dtype=tf.float32, name=None))


In [3]:
img=Image.open(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Train\Train\0\3.png')
img.show()

In [3]:
train_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Train\Train')
test_dir = pathlib.Path(r'C:\Users\ethan\Downloads\mandarin_images\new_test\final_images\CASIA-HWDB_Test\Test')
'''
train_chinese = os.listdir(train_dir)
test_chinese = os.listdir(test_dir)
len(train_chinese)
len(test_chinese)
'''

'\ntrain_chinese = os.listdir(train_dir)\ntest_chinese = os.listdir(test_dir)\nlen(train_chinese)\nlen(test_chinese)\n'

In [4]:
def maximize_contrast(image, label):
    # Rescale image values to [0, 1]
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    # Maximize contrast using tf.image.adjust_contrast
    image = tf.image.adjust_contrast(image, contrast_factor=2.0)
    
    return image, label


In [5]:
#make tensorflow dataset from image directory


train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    seed=42,
    validation_split = 0.2,
    subset = 'both',
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = 32,
    image_size = (56, 56),
    shuffle = True,
    interpolation = "bilinear",
    crop_to_aspect_ratio = False
)



Found 3223042 files belonging to 7330 classes.
Using 2578434 files for training.
Using 644608 files for validation.
Found 776523 files belonging to 7330 classes.


In [6]:
#apply max contrast to images

train_dataset = train_dataset.map(maximize_contrast)
val_dataset = val_dataset.map(maximize_contrast)
test_dataset =  test_dataset.map(maximize_contrast)

print("starting rescaling")

rescaling_layer = layers.Rescaling(scale=1./255)
train_dataset = train_dataset.map(lambda x, y: (rescaling_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (rescaling_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (rescaling_layer(x), y))


starting rescaling


In [11]:

print("starting normalization")
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

'''
train_dataset_for_adapt = train_dataset.map(lambda x, y: x)
test_dataset_for_adapt = test_dataset.map(lambda x, y: x)
val_dataset_for_adapt = val_dataset.map(lambda x, y: x)

normalization_layer_train = layers.Normalization(axis=-1)
normalization_layer_test = layers.Normalization(axis=-1)
normalization_layer_val = layers.Normalization(axis=-1)
'''

def calculate_mean_variance_incrementally(dataset):
    # Initialize variables for mean, variance, and sample count
    mean_accumulator = 0.0
    variance_accumulator = 0.0
    sample_count = 0
    
    for images, _ in dataset:
        # Flatten images to process all pixels
        flattened_images = tf.reshape(images, (images.shape[0], -1))
        batch_size, num_pixels = flattened_images.shape
        
        # Update total sample count
        sample_count += batch_size * num_pixels
        
        # Calculate batch mean and variance
        batch_mean = tf.reduce_mean(flattened_images)
        batch_variance = tf.reduce_mean(tf.square(flattened_images - batch_mean))
        
        # Incrementally update the accumulated mean and variance
        delta = batch_mean - mean_accumulator
        mean_accumulator += delta / sample_count * (batch_size * num_pixels)
        variance_accumulator += batch_variance * (batch_size * num_pixels)
    
    # Finalize mean and variance calculation
    overall_mean = mean_accumulator
    overall_variance = variance_accumulator / sample_count
    
    return overall_mean.numpy(), overall_variance.numpy()


train_mean, train_variance = calculate_mean_variance_incrementally(train_dataset)
test_mean, test_variance = calculate_mean_variance_incrementally(test_dataset)
val_mean, val_variance = calculate_mean_variance_incrementally(val_dataset)

normalization_layer_train = layers.Normalization(axis=-1, mean=train_mean, variance=train_variance)
normalization_layer_test = layers.Normalization(axis=-1, mean=test_mean, variance=test_variance)
normalization_layer_val = layers.Normalization(axis=-1, mean=val_mean, variance=val_variance)

'''
print("starting adapt")
normalization_layer_train.adapt(train_dataset_for_adapt)
normalization_layer_test.adapt(test_dataset_for_adapt)
normalization_layer_val.adapt(val_dataset_for_adapt)

print("finished adapt")

'''
'''
train_dataset = train_dataset.map(lambda x, y: (normalization_layer_train(x), y), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer_test(x), y), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: (normalization_layer_val(x), y), num_parallel_calls=tf.data.AUTOTUNE)

'''

starting normalization


ResourceExhaustedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to allocate memory for the batch of component 1 [Op:IteratorGetNext]

In [7]:
print("starting augmentation")

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))
test_dataset = test_dataset.map(lambda x, y: (data_augmentation(x), y))
val_dataset = val_dataset.map(lambda x, y: (data_augmentation(x), y))

starting augmentation


In [8]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

model = tf.keras.Sequential()
#model.add(normalization_layer_train)
model.add(layers.Conv2D(filters = 80, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(56,56,1)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters= 160, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters= 320, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters= 640, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dense(units=200, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(units=7330, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 80)        800       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 80)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 160)       115360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 160)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 320)       461120    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 320)       

In [11]:
num_classes = train_dataset.num_classes
print(num_classes)

AttributeError: 'MapDataset' object has no attribute 'num_classes'